In [2]:
import jax 
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import jaxopt

In [111]:
# Maybe not giving enough constraints? The inequalities are equalities after optimization through the multipliers...
# The dual fulfills the constraints, but do not optimize the objective function...
# Do you typically enforces to be dual feasible? + the constraints are not always respected...


# Nvm, but really need to see how the penalty method is working here 
@jax.jit
def f(params, k):
    Q_0 = jnp.array([[1, 0], [0, 9]])
    P_1 = -jnp.array([2, 1])
    P_2 = -jnp.array([1, 3])
    P_3 = -jnp.array([1, 0])
    P_4 = -jnp.array([0, 1])
    P = params[0]*P_1 + params[1]*P_2 + params[2]*P_3 + params[3]*P_4
    c_1 = 1
    c_2 = 1
    c = params[0]*c_1 + params[1]*c_2
    x = -0.5*jnp.linalg.inv(Q_0)@P
    D = x.T@Q_0@x + P@x + c
    return (-D + k*(jnp.sum((params**2)*(params<0))))

In [3]:
og_params = jnp.array([1., 1., 1., 1.]).astype("float32")

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [28]:
t = jnp.linspace(0, 5, 101)
mesh = np.zeros((101, 101))
mesh_2 = np.zeros((101,101))
for i in range(len(t)):
    for j in range(len(t)):
        mesh[i][j] = f(jnp.array([t[i], t[j]]))[0]
        mesh_2[i][j] = f(jnp.array([t[i], t[j]]))[1]

In [47]:
jnp.where((mesh_2>-0.01)*(mesh_2<0.01)*(mesh<0.01)), f(jnp.array([t[1], t[19]]))

((Array([ 0,  1,  5,  6,  7, 11, 12, 13, 17], dtype=int32),
  Array([20, 19, 14, 13, 12,  7,  6,  5,  0], dtype=int32)),
 (Array(0.4907639, dtype=float32),
  Array([0.525    , 0.1611111], dtype=float32)))

In [112]:
optimizer = jaxopt.GradientDescent(f, maxiter = 1)
params = og_params.copy()
for i in range(15):
    for j in range(20):
        print(-f(params, 0))
        params, state = optimizer.run(params, (2**i)*0.001)

-2.6944442
0.022322893
0.17127487
0.3745765
0.48039317
0.63746893
0.745577
0.8408406
0.99166137
1.0936631
1.2196201
1.315411
1.477584
1.6069465
1.6967704
1.8589411
1.991732
2.0762782
2.2385418
2.3743923
2.4823372
2.5727985
2.7119987
2.8112998
2.9015355
3.0415819
3.1400387
3.2260828
3.3718634
3.46341
3.5913258
3.672005
3.824825
3.9554155
4.031134
4.1833267
4.316163
4.4128246
4.5015564
4.6336164
4.7314634
4.805916
4.93174
5.0112896
5.090705
5.208804
5.2955217
5.36586
5.4941745
5.5672297
5.682324
5.745104
5.8721642
5.988129
6.046793
6.1722918
6.2888064
6.343771
6.4676943
6.584476
6.636114
6.706106
6.7638164
6.802194
6.887428
6.9245696
6.97311
7.042255
7.09451
7.130741
7.21252
7.247155
7.3259473
7.349928
7.420669
7.4977145
7.5201464
7.588852
7.6641135
7.6851397
7.751861
7.7234335
7.690581
7.669589
7.6598563
7.629554
7.61031
7.6011834
7.5730696
7.555289
7.5467052
7.5205164
7.5040164
7.4959335
7.471491
7.464376
7.439504
7.4343476
7.4082923
7.4076347
7.391004
7.08554
7.083322
6.741595
6.81620

In [113]:
params

Array([-0.00470802,  1.015672  , -0.01504018, -0.00514522], dtype=float32)

In [114]:
Q = jnp.array([[1, 0], [0, 9]])
P = jnp.array([[2, 1],[1, 3]])
ideal_x = jnp.array([0.5, 1/6])
Q_0 = jnp.array([[1, 0], [0, 9]])
P_1 = -jnp.array([2, 1])
P_2 = -jnp.array([1, 3])
P_3 = -jnp.array([1, 0])
P_4 = -jnp.array([0, 1])
P = params[0]*P_1 + params[1]*P_2 + params[2]*P_3 + params[3]*P_4
x = -0.5*jnp.linalg.inv(Q_0)@P
x.T@Q_0@x, ideal_x.T@Q_0@ideal_x, x, ideal_x

(Array(0.5018593, dtype=float32),
 Array(0.5, dtype=float32),
 Array([0.4956079 , 0.16873126], dtype=float32),
 Array([0.5       , 0.16666667], dtype=float32))

In [55]:
1 - jnp.array([2, 1])@x, 1 - jnp.array([1, 3])@x

(Array(-0.02247119, dtype=float32), Array(0.08356863, dtype=float32))